# Creating Red Team Scenarios

In [ ]:
from ollama import Client
import os
from dotenv import load_dotenv
from mitreattack.stix20 import MitreAttackData

# Load environment variables from .env file
load_dotenv()

In [ ]:
client = Client(host=os.getenv("OLLAMA_CHAT"))
mitre_attack_data = MitreAttackData("enterprise-attack.json")

In [ ]:
# Extract keywords from the description
def extract_keywords_from_description(description):
    # Define the merged prompt
    prompt = (f"Given the cybersecurity scenario description: '{description}', identify and list the key terms, "
              "techniques, or technologies relevant to MITRE ATT&CK. Extract TTPs from the scenario. "
              "If the description is too basic, expand upon it with additional details, applicable campaign, "
              "or attack types based on dataset knowledge. Then, extract the TTPs from the revised description.")
    # Set up the messages for the Ollama API
    messages = [
        {
            "role": "system",
            "content": "You are a cybersecurity professional with more than 25 years of experience."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    # Set the request options
    options = {
        "temperature": 0.7
    }
    # Make the API call
    try:
        client = Client(host=os.getenv("OLLAMA_CHAT"))
        response = client.chat(
            model="llama3",
            messages=messages,
            options=options,
            stream=False            
        )
        response_content = response['message']['content'].strip()
        keywords = response_content.split(', ')
        return keywords
    except Exception as e:
        print("An error occurred while connecting to the OpenAI API:", e)
        return []

In [ ]:
# Generate a comprehensive scenario
def generate_ttp_chain(match):
    # Create a prompt for GPT-3 to generate a TTP chain for the provided match
    prompt = (f"Given the MITRE ATT&CK technique '{match['name']}' and its description '{match['description']}', "
              "generate an example scenario and TTP chain demonstrating its use.")
    # Set up the messages for the OpenAI API
    messages = [
        {
            "role": "system",
            "content": "You are a cybersecurity professional with expertise in MITRE ATT&CK techniques."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    # Set the request options
    options = {
        "temperature": 0.7
    }
    # Make the API call
    try:
        client = Client(host=os.getenv("OLLAMA_CHAT"))
        response = client.chat(
            model="llama3",
            messages=messages,
            options=options,
            stream=False            
        )
        response_content = response['message']['content'].strip()
        return response_content
    except Exception as e:
        print("An error occurred while generating the TTP chain:", e)
        return "Unable to generate TTP chain."

In [ ]:
# Search the ATT&Ck dataset for extracted keywords
def score_matches(matches, keywords):
    scores = []
    for match in matches:
        score = sum([keyword in match['name'] for keyword in keywords]) + \
                sum([keyword in match['description'] for keyword in keywords])
        scores.append((match, score))
    return scores
def search_dataset_for_matches(keywords):
    matches = []
    for item in mitre_attack_data.get_techniques():
        if any(keyword in item['name'] for keyword in keywords):
            matches.append(item)
        elif 'description' in item and any(keyword in item['description'] for keyword in keywords):
            matches.append(item)
    return matches

In [ ]:
#description = input("Enter your scenario description: ")
description = "The scenario is this: an electic utility is being attacked by APT29."
keywords = extract_keywords_from_description(description)
matches = search_dataset_for_matches(keywords)
scored_matches = score_matches(matches, keywords)
# Sort by score in descending order and take the top 3
top_matches = sorted(scored_matches, key=lambda x: x[1], reverse=True)[:3]

In [ ]:
print("Writing top 3 matches from the MITRE ATT&CK dataset:")
with open("ollama_red_team_scenarios.txt", "w") as file:
    for match, score in top_matches:
        file.write("Name:" + match['name'] +"\n")
        file.write("Summary:" + match['description'] + "\n")
        ttp_chain = generate_ttp_chain(match)
        file.write("Example Scenario and TTP Chain:" + ttp_chain + "\n")
        file.write("-" * 50 + "\n")

# Google Dork Automation

In [ ]:
from ollama import Client
import os
from dotenv import load_dotenv
import requests
import time

# Load environment variables from .env file
load_dotenv()

In [ ]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
CSE_ID = os.getenv("CSE_ID")
SEARCH_URL = "https://www.googleapis.com/customsearch/v1?q={query}&key={api_key}&cx={cse_id}"

In [ ]:

prompt = ("You are a cybersecurity professional specializing in red team tactics." 
          "I am a cybersecurity professional and I have a scenario where I need to find exposed documents on a my own domain." 
          "Please provide a list of example Google dorks that I can use to discover such vulnerabilities" 
          "as part of an authorized exercise on my own authorized domain."
          "At the end of the reponse, include the dorks in a Python list"          
          )
client = Client(host=os.getenv("OLLAMA_CHAT"))
response = client.chat(
    model='llama3',
    messages=[{'role': 'user', 'content': prompt}],
    stream=False
)
print(response['message']['content'])

In [ ]:
dorks = [
    "site:{} \"intranet\" -filetype:pdf".format("yourdomain.com"),
    "site:{} \"confidential\" OR \"proprietary\" -filetype:pdf".format("yourdomain.com"),
    "site:{} \"news\" \"archive\" -filetype:pdf".format("yourdomain.com"),
    "site:{} \"HR\" OR \"finance\" OR \"security\" -inurl:www".format("yourdomain.com")
]
dorks_modified = [dork.replace("yourdomain.com", "fpl.com") for dork in dorks]

In [ ]:
def get_search_results(query):
    """Fetch the Google search results."""
    response = requests.get(SEARCH_URL.format(query=query, api_key=GOOGLE_API_KEY, cse_id=CSE_ID))
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code)
        return {}

In [ ]:
with open("ollama_dork_results.txt", "a") as outfile:
    for dork in dorks_modified:
        print(f"Running dork: {dork}")
        results = get_search_results(dork)
        if 'items' in results:
            for item in results['items']:
                print(item['title'])
                print(item['link'])
                outfile.write(item['title'] + "\n")
                outfile.write(item['link'] + "\n")
                outfile.write("-" * 50 + "\n")
        else:
            print("No results found or reached API limit!")
        # To not hit the rate limit, introduce a delay between requests
        time.sleep(20)

# LLM-powered Terminals

> While it works, this section is better off in a script file than a notebook

In [ ]:
from ollama import Client
import os
from dotenv import load_dotenv
import subprocess

# Load environment variables from .env file
load_dotenv()

In [ ]:
def open_file(filepath): #Open and read a file
    with open(filepath, 'r', encoding='UTF-8') as infile:
        return infile.read()
def save_file(filepath, content): #Create a new file or overwrite an existing one.
    with open(filepath, 'w', encoding='UTF-8') as outfile:
        outfile.write(content)
def append_file(filepath, content): #Create a new file or append an existing one.
    with open(filepath, 'a', encoding='UTF-8') as outfile:
        outfile.write(content)

In [ ]:
def llama3(prompt):
    try:
        client = Client(host=os.getenv("OLLAMA_CHAT"))
        response = client.chat(
            model='llama3',
            messages=[{'role': 'user', 'content': prompt}],
            stream=False
        )
        text = response['message']['content'].strip().replace("`","")
        return text
    except Exception as e:
        print(f"\nError communicating with the API.")
        print(f"\nError: {e}")
        print("\nRetrying...")
        return llama3(prompt)

In [ ]:
while True:
    request = input("\nEnter request: ")
    if not request:
        break
    if request == "quit":
        break
    prompt = open_file("prompt_linux_cli.txt").replace('{INPUT}', request)
    command = llama3(prompt)
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, bufsize=1, universal_newlines=True)
    print("\n" + command + "\n")
    with process:
        for line in process.stdout:
            print(line, end='', flush=True)
    exit_code = process.wait()
    append_file("command-log.txt", "Request: " + request + "\nCommand: " + command + "\n\n") #Write the request and GPT generated command to a log